In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

import cv2
import numpy as np

def get_background(file_path):
    cap = cv2.VideoCapture(file_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Handle frames with different dimensions, if necessary
        # frame = cv2.resize(frame, (desired_width, desired_height))

        frames.append(frame)

    cap.release()  # Release the video capture object
    if frames:
        median_frame = np.median(frames, axis=0).astype(np.uint8)
        return median_frame
    else:
        # Handle the case when no frames are read
        print("Error: No frames were read from the video.")
        return None

# Example usage
background = get_background("HethenHisPelivideo.mp4")

if background is not None:
    print("Background data type:", background.dtype)
    background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    print("Background data type after conversion:", background_gray.dtype)
else:
    print("Error: No valid background frame.")



def ORB_detector(new_image, image_template):
    orb = cv2.ORB_create(1000, 1.2)  # ORB detector of 1000 keypoints, scaling pyramid factor=1.2
    (kp1, des1) = orb.detectAndCompute(new_image, None)  # Detect keypoints on the new image
    (kp2, des2) = orb.detectAndCompute(image_template, None)  # Detect keypoints of the template image
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)  # Matcher
    matches = bf.match(des1, des2)  # Extract matches
    matches = sorted(matches, key=lambda val: val.distance)  # Sort matches
    img2 = cv2.drawKeypoints(image_template, kp2, None, color=(0,255,0), flags=0)
    img1 = cv2.drawKeypoints(new_image, kp2, None, color=(0,255,0), flags=0)
    plt.imshow(img2)
    plt.show()
    return len(matches)

# Load video file and template image

consecutive_frames = 4

background = get_background("He&HisPelivideo.mp4")
print("Background data type:", background.dtype)
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)


frame_count = 0
frame_diff_list = []

while True:
    cap = cv2.VideoCapture("He&HisPelivideo.mp4")
    ret, frame = cap.read()

    frame_count += 1
    orig_frame = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.uint8)
    print("Background shape:", background.shape)
    print("Gray shape:", gray.shape)

    frame_diff = cv2.absdiff(gray, background_gray)
    ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
    dilate_frame = cv2.dilate(thres, None, iterations=2)
    frame_diff_list.append(dilate_frame)
    if len(frame_diff_list) == consecutive_frames:
        sum_frames = sum(frame_diff_list)
        contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) < 500:
                continue
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(orig_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Get height and width of webcam frame
        height, width = frame.shape[:2]

        # Define ROI Box Dimensions (Note some of these things should be outside the loop)
        top_left_x = int(width / 3)
        top_left_y = int((height / 2) + (height / 4))
        bottom_right_x = int((width / 3) * 2)
        bottom_right_y = int((height / 2) - (height / 4))

        # Draw rectangular window for our region of interest
        # cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), 255, 3)

        # Crop window of observation we defined above
        cropped = frame[bottom_right_y:top_left_y , top_left_x:bottom_right_x]

        # Flip frame orientation horizontally
        frame = cv2.flip(frame,1)

        # Get number of ORB matches 
        matches = ORB_detector(cropped, image_template)

        # Display status string showing the current no. of matches 
        output_string = "Matches = " + str(matches)
        cv2.putText(frame, output_string, (50,450), cv2.FONT_HERSHEY_COMPLEX, 2, (250,0,150), 2)

        # Our threshold to indicate object deteciton
        # For new images or lightening conditions you may need to experiment a bit 
        # Note: The ORB detector to get the top 1000 matches, 350 is essentially a min 35% match
        threshold = 336

        # If matches exceed our threshold then object has been detected
        if matches > threshold:
            cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
            cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)
            # Display the frame with the detected object
        cv2.imshow('Object Detector using ORB', frame)

        cv2.imshow('Detected Objects', orig_frame)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

def HSV_detector(path_to_image):
    image = cv2.imread(path_to_image)
    # Convert the image from BGR to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define lower and upper bounds for dark red color in HSV
    lower_dark_red = np.array([0, 50, 50])
    upper_dark_red = np.array([10, 255, 255])

    # Define lower and upper bounds for yellow color in HSV
    lower_yellow = np.array([20, 100, 50])
    upper_yellow = np.array([40, 255, 255])

    # Define lower and upper bounds for green-blue color in HSV
    lower_green_blue = np.array([90, 100, 50])
    upper_green_blue = np.array([130, 255, 255])

    # Threshold the image to extract dark red objects
    dark_red_mask = cv2.inRange(hsv, lower_dark_red, upper_dark_red)

    # Threshold the image to extract yellow objects
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Threshold the image to extract green-blue objects
    green_blue_mask = cv2.inRange(hsv, lower_green_blue, upper_green_blue)

    # Combine the masks to obtain the final mask
    final_mask = cv2.bitwise_or(cv2.bitwise_or(dark_red_mask, yellow_mask), green_blue_mask)

    # Bitwise-AND the original image and the final mask to extract the objects
    extracted_objects = cv2.bitwise_and(image, image, mask=final_mask)

    return extracted_objects

# Example usage
path_to_image = 'Pelivideo.png'
result = HSV_detector(path_to_image)

# Display the original and extracted images
cv2.imshow('Original Image', cv2.imread(path_to_image))
cv2.imshow('Extracted Objects', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# Version 1.1 20.32 10.11.2023

# Steps:
'''
- Screenshot a frame in the video with the presence of the object
- Use HSV_detector to filter out the content of the object from the background
- Put it into ORB_detector with the current frame
- Draw the bounding for the object from frame-to-frame
'''

import cv2
import numpy as np
from matplotlib import pyplot as plt

cap = cv2.VideoCapture("HethenHisPelivideo.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
target_time = 4
init_frame = int(target_time * fps)

initial_frame = cv2.imread("Pelivideo.png")
gray_initial_frame = cv2.cvtColor(initial_frame, cv2.COLOR_BGR2GRAY)


while True:
    
    ret, frame = cap.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    blur0 = cv2.GaussianBlur(gray_initial_frame, (5, 5), 0)
    blur1 = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    
    edges0 = cv2.Canny(blur0,10,50)
    edges1 = cv2.Canny(blur1,10,50)
    
    highlighted_image = np.copy(frame)
    contours, _ = cv2.findContours(edges0, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(highlighted_image, contours, -1, (0, 255, 0), 2)
    
    orb = cv2.ORB_create(1000, 1.2, 2, 31)  # ORB detector of 1000 keypoints, scaling pyramid factor=1.2
    (kp1, des1) = orb.detectAndCompute(edges0, None)  # Detect keypoints on the new image
    (kp2, des2) = orb.detectAndCompute(edges1, None)  # Detect keypoints of the template image
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)  # Matcher
    matches = bf.match(des1, des2)  # Extract matches
    matches = sorted(matches, key=lambda val: val.distance)  # Sort matches
    
    output_string = "Matches = " + str(len(matches))
    cv2.putText(frame, output_string, (50,450), cv2.FONT_HERSHEY_COMPLEX, 2, (250,0,150), 2)
    
    # Detection layer-into-layer
    initial_frame = frame
    kp1 = kp2
    
    threshold = 280
    if len(matches) > threshold:
        cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)
        for match in matches[:100]:
            query_idx = match.queryIdx
            if query_idx < len(kp1):
                x, y = kp1[query_idx].pt
                cv2.rectangle(frame, (int(x + 20), int(y + 20)), (int(x + 10), int(y + 10)), (0, 255, 0), 2)
                cv2.imshow("Object Detecting", frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.imshow('Original Image', frame)
cv2.imshow('Canny Edges', edges)
cv2.imshow('Highlighted Edges', highlighted_image)
        
cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np


def ORB_detector(new_image, image_template):
    # Function that compares input image to template
    # It then returns the number of ORB matches between them
    image1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Create ORB detector with 1000 keypoints with a scaling pyramid factor of 1.2
    orb = cv2.ORB_create(1000, 1.2)

    # Detect keypoints of original image
    (kp1, des1) = orb.detectAndCompute(image1, None)

    # Detect keypoints of rotated image
    (kp2, des2) = orb.detectAndCompute(image_template, None)

    # Create matcher 
    # Note we're no longer using Flannbased matching
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    # Do matching
    matches = bf.match(des1,des2)

    # Sort the matches based on distance.  Least distance
    # is better
    matches = sorted(matches, key=lambda val: val.distance)
    return len(matches)

cap = cv2.VideoCapture("HethenHisPelivideo.mp4")

# Load our image template, this is our reference image
image_template = cv2.imread('Pelivideo.png', 0) 
# image_template = cv2.imread('images/kitkat.jpg', 0) 

while True:
    # Get webcam images
    ret, frame = cap.read()

    # Get height and width of webcam frame
    height, width = frame.shape[:2]

    # Define ROI Box Dimensions (Note some of these things should be outside the loop)
    top_left_x = int(width / 3)
    top_left_y = int((height / 2) + (height / 4))
    bottom_right_x = int((width / 3) * 2)
    bottom_right_y = int((height / 2) - (height / 4))

    # Draw rectangular window for our region of interest
    cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), 255, 3)

    # Crop window of observation we defined above
    cropped = frame[bottom_right_y:top_left_y , top_left_x:bottom_right_x]

    # Flip frame orientation horizontally
    frame = cv2.flip(frame,1)

    # Get number of ORB matches 
    matches = ORB_detector(cropped, image_template)

    # Display status string showing the current no. of matches 
    output_string = "Matches = " + str(matches)
    cv2.putText(frame, output_string, (50,450), cv2.FONT_HERSHEY_COMPLEX, 2, (250,0,150), 2)

    # Our threshold to indicate object deteciton
    # For new images or lightening conditions you may need to experiment a bit 
    # Note: The ORB detector to get the top 1000 matches, 350 is essentially a min 35% match
    threshold = 250

    # If matches exceed our threshold then object has been detected
    if matches > threshold:
        cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
        cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)

    cv2.imshow('Object Detector using ORB', frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
rect = cv2.minAreaRect(points)

    # Extract the rotation angle and bounding box
    angle = rect[2]
    box = cv2.boxPoints(rect)
    box = np.int0(box)

    # Rotate the image and bounding box
    rotated_image = image.copy()
    rotated_image = cv2.warpAffine(rotated_image, cv2.getRotationMatrix2D(rect[0], angle, 1.0), (image.shape[1], image.shape[0]))
    box = cv2.transform(np.array([box]), cv2.getRotationMatrix2D(rect[0], angle, 1.0)).squeeze()

    # Draw a rotated rectangle around the region with more than 50 keypoints
    cv2.drawContours(rotated_image, [box], 0, (0, 255, 0), 2)

    
    